<a href="https://colab.research.google.com/github/tolgakurtuluss/HacktoberFest/blob/main/aiornot_works.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preliminaries

In [1]:
!pip install -q transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 88.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 101.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 15.3 MB/s eta 0:00:00


In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid.
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as d

In [3]:
!git config --global credential.helper store

In [4]:
from huggingface_hub import hf_hub_download
from shutil import unpack_archive
from pathlib import Path
import pandas as pd

train_zip_file = hf_hub_download('competitions/aiornot', 'train.zip', repo_type='dataset', revision='b843a82bd712648b2fe0dc043cf8a04475491d38')
unpack_archive(train_zip_file, './')

test_zip_file = hf_hub_download('competitions/aiornot', 'test.zip', repo_type='dataset', revision='b843a82bd712648b2fe0dc043cf8a04475491d38')
unpack_archive(test_zip_file, './')

dataframe = pd.read_csv(hf_hub_download('competitions/aiornot', 'train.csv', repo_type='dataset', revision='b843a82bd712648b2fe0dc043cf8a04475491d38'))

## Data splitting

In [5]:
dataframe.head()

,id,label
0,0.jpg,1
1,1.jpg,1
2,2.jpg,1
3,3.jpg,0
4,4.jpg,1


In [6]:
dataframe["label"].value_counts()

1    10330
0     8288
Name: label, dtype: int64

In [7]:
from sklearn.model_selection import train_test_split 

seed= 42
train_df, validation_df = train_test_split(
    dataframe, test_size=0.1, stratify=dataframe["label"].values, random_state=seed
)
validation_df, holdout_df = train_test_split(
    validation_df, test_size=0.5, stratify=validation_df["label"].values, random_state=seed
)

train_df = train_df.reset_index(drop=True)
validation_df = validation_df.reset_index(drop=True)
holdout_df = holdout_df.reset_index(drop=True)

train_df.shape, validation_df.shape, holdout_df.shape

((16756, 2), (931, 2), (931, 2))

In [8]:
train_df["image_path"] = train_df["id"].apply(lambda x: "train/" + x)
validation_df["image_path"] = validation_df["id"].apply(lambda x: "train/" + x)
holdout_df["image_path"] = holdout_df["id"].apply(lambda x: "train/" + x)

In [9]:
from imutils import paths
import pandas as pd 

test_image_paths = list(paths.list_images("test"))
final_test_df = pd.DataFrame()
final_test_df["image_path"] = test_image_paths
final_test_df["label"] = 0
final_test_df.head()

,image_path,label
0,test/14965.jpg,0
1,test/39665.jpg,0
2,test/706.jpg,0
3,test/17862.jpg,0
4,test/32337.jpg,0


## Data prep

In [10]:
import torch 
import PIL

class AIOrNotDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, transforms, split):
        self.dataframe = dataframe
        self.transforms = transforms 
        self.split = split

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        img_path = self.dataframe.loc[idx, "image_path"]
        image = PIL.Image.open(img_path)
        image = self.transforms(image)
        
        if self.split != "test":
            label = self.dataframe.loc[idx, "label"] 
            return {"img_path": img_path, "image": image, "label": label} 
        else:
            return {"img_path": img_path, "image": image}

In [11]:
from transformers import AutoModelForImageClassification, AutoImageProcessor


label2id = {"no_ai": 0, "ai": 1}
id2label = {0: "no_ai", 1: "ai"}

#model_ckpt = "microsoft/beit-base-patch16-224-pt22k-ft22k" ++v3

model_ckpt = "nvidia/mit-b0"
#model_ckpt = "timm/convnextv2_nano.fcmae_ft_in22k_in1k"

model = AutoModelForImageClassification.from_pretrained(
    model_ckpt, 
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes = True, # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
)
image_processor = AutoImageProcessor.from_pretrained(model_ckpt)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Some weights of SegformerForImageClassification were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 256]) in the checkpoint and torch.Size([2, 256]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
/usr/local/lib/python3.8/dist-packages/transformers/models/segformer/image_processing_segformer.py:102: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


In [32]:
from torchvision import transforms as T


mean, std = image_processor.image_mean, image_processor.image_std
size = image_processor.size["height"]

train_transforms = T.Compose([
    T.RandomResizedCrop(size),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize(mean=mean, std=std)
])
test_transforms = T.Compose([
    T.Resize(256),
    T.CenterCrop(size),
    T.ToTensor(),
    T.Normalize(mean=mean, std=std)
])

In [33]:
train_dataset = AIOrNotDataset(train_df, train_transforms, split="train")
val_dataset = AIOrNotDataset(validation_df, test_transforms, split="validation")
holdout_dataset = AIOrNotDataset(holdout_df, test_transforms, split="holdout")
final_test_dataset = AIOrNotDataset(final_test_df, test_transforms, split="test")

In [34]:
sample_train = next(iter(train_dataset))
sample_train.keys(), sample_train["image"].size()

(dict_keys(['img_path', 'image', 'label']), torch.Size([3, 512, 512]))

## Prepare trainer

In [55]:
from transformers import TrainingArguments

model_name = model_ckpt.split("/")[-1]
batch_size = 64

args = TrainingArguments(
    f"{model_name}-finetuned-aiornot-test-1",
    remove_unused_columns=False,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=3,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [56]:
from sklearn.metrics import accuracy_score, f1_score, log_loss

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    
    f1 = f1_score(labels, preds, average="macro")
    acc = accuracy_score(labels, preds)
    logloss = log_loss(labels, preds)
    return {"accuracy": acc, "f1": f1, "log_loss": logloss}

In [57]:
def collate_fn(examples):
    pixel_values = torch.stack([example["image"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [58]:
from transformers import Trainer 

trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)

/content/mit-b0-finetuned-aiornot-test-1 is already a clone of https://huggingface.co/tkurtulus/mit-b0-finetuned-aiornot-test-1. Make sure you pull the latest changes with `repo.git_pull()`.


## Train!

In [59]:
import torch
torch.cuda.empty_cache()

torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 7            |        cudaMalloc retries: 7         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   13725 MB |   13773 MB |   24920 MB |   11195 MB |\n|       from large pool |   13698 MB |   13746 MB |   24844 MB |   11146 MB |\n|       from small pool |      27 MB |      28 MB |      76 MB |      49 MB |\n|---------------------------------------------------------------------------|\n| Active memory         |   13725 MB |   13773 MB |   24920 MB |   11195 MB |\n|       from large pool |   13698 MB |   13746 MB |

In [60]:
train_results = trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 16756
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 192
  Gradient Accumulation steps = 3
  Total optimization steps = 261
  Number of trainable parameters = 3319906


OutOfMemoryError: ignored

In [36]:
trainer.evaluate(holdout_dataset)

***** Running Evaluation *****
  Num examples = 931
  Batch size = 64


{'eval_loss': 0.06090405210852623,
 'eval_accuracy': 0.9763694951664876,
 'eval_f1': 0.9760287439726605,
 'eval_log_loss': 0.8161824642824049,
 'eval_runtime': 20.3491,
 'eval_samples_per_second': 45.751,
 'eval_steps_per_second': 0.737,
 'epoch': 9.99}

## Prepare submission file

In [23]:
test_loader = torch.utils.data.DataLoader(
    final_test_dataset, batch_size=256, pin_memory=True
)
device = "cuda" if torch.cuda.is_available() else "cpu"
_ = model.to(device)

In [24]:
file_paths = []
pred_ids = []

for batch in test_loader:
    image_paths = batch["img_path"]
    image_paths = [x.split("/")[-1] for x in image_paths]
    file_paths.extend(image_paths)
    
    images = batch["image"].to(device)
    inputs = {"pixel_values": images}
    
    with torch.no_grad(): 
        logits = model(**inputs).logits

    predictions = logits.argmax(-1).cpu().numpy().tolist()
    pred_ids.extend(predictions)

In [25]:
submission_df = pd.DataFrame({"id": file_paths, "label": pred_ids})
submission_df.head()

,id,label
0,14965.jpg,0
1,39665.jpg,1
2,706.jpg,1
3,17862.jpg,1
4,32337.jpg,0


In [26]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

submission_df.to_csv(f"{TIMESTAMP}.csv", index=False)

In [27]:
from google.colab import files
files.download(f"{TIMESTAMP}.csv") 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [30]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:10024"

In [ ]:
#################################

from transformers import AutoFeatureExtractor, AutoModelForImageClassification

extractor = AutoFeatureExtractor.from_pretrained("tkurtulus/beit-base-patch16-224-pt22k-ft22k-finetuned-aiornot-test-3")

model = AutoModelForImageClassification.from_pretrained("tkurtulus/beit-base-patch16-224-pt22k-ft22k-finetuned-aiornot-test-3")